# Installing preliminary dependencies

In [1]:
import time
from pymata4 import pymata4

import face_recognition
import imutils
import pickle
import cv2
import os

import smtplib
import imghdr
from email.message import EmailMessage # EmailMessage class helps to create a template for an email message

from pushbullet import Pushbullet

# Prerequisites of face recognition

In [5]:
#find path of xml file containing haarcascade file 
cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved in last file
data = pickle.loads(open('face_enc', "rb").read())

directory = r'D:\AI Projects\New_Intruder'

# Prerequisites of sending email through python

In [3]:
EMAIL_ADDR = 'prajwalnayak2000@gmail.com'
EMAIL_PASS = 'ldltqmpsmosxdjou'

In [4]:
msg = EmailMessage()
msg['Subject'] = 'Unknown Entry !!'
msg['From'] = EMAIL_ADDR
msg['To'] = EMAIL_ADDR
msg.set_content('Unknown image attached...')

# Prerequisites of sending notification through python

In [5]:
API_KEY = "o.QLICL6EDs86MVmUT2c0u8E8X0wxncQ8x"
file="notify/message.txt"

with open(file,mode='r') as f:
    text=f.read()

pb=Pushbullet(API_KEY)

# Functions

In [2]:
def send_email(e):
#     push=pb.push_note('Alert!! Unknown Enrty',text)
    with open("intruder_{}.jpg".format(e),'rb') as f:
        file_data = f.read()
        file_type = imghdr.what(f.name)
        file_name =f.name

    msg.add_attachment(file_data,maintype = 'image', subtype = file_type, filename = file_name)
    
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(EMAIL_ADDR,EMAIL_PASS)
        smtp.send_message(msg)

In [7]:
def take_image():
    print("Taking images!!")
    video_capture = cv2.VideoCapture(0)
    os.chdir(directory)
    for k in range(3):
        # grab the frame from the threaded video stream
        ret, frame = video_capture.read()
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        filename='intruder_{}.jpg'.format(k)
        cv2.imwrite(filename, frame)
        cv2.imshow("Frame", frame)
        send_email(k)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break    
    video_capture.release()
    cv2.destroyAllWindows()

In [6]:
def face_recog():
    print("Taking images!!")
    video_capture = cv2.VideoCapture(0)
    os.chdir(directory)
    # loop over frames from the video file stream
    for k in range(3):
        # grab the frame from the threaded video stream
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(60, 60),
                                             flags=cv2.CASCADE_SCALE_IMAGE)
     
        # convert the input frame from BGR to RGB 
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # the facial embeddings for face in input
        encodings = face_recognition.face_encodings(rgb)
        names = []
        # loop over the facial embeddings incase
        # we have multiple embeddings for multiple fcaes
        for encoding in encodings:
           #Compare encodings with encodings in data["encodings"]
           #Matches contain array with boolean values and True for the embeddings it matches closely
           #and False for rest
            matches = face_recognition.compare_faces(data["encodings"],
             encoding)
            #set name =inknown if no encoding matches
            name = "Unknown"
            # check to see if we have found a match
            if True in matches:
                #Find positions at which we get True and store them
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}
                # loop over the matched indexes and maintain a count for
                # each recognized face face
                for i in matchedIdxs:
                    #Check the names at respective indexes we stored in matchedIdxs
                    name = data["names"][i]
                    #increase count for the name we got
                    counts[name] = counts.get(name, 0) + 1
                #set name which has highest count
                name = max(counts, key=counts.get)
     
     
            # update the list of names
            names.append(name)
            # loop over the recognized faces
            for ((x, y, w, h), name) in zip(faces, names):
                # rescale the face coordinates
                # draw the predicted face name on the image
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                 0.75, (0, 255, 0), 2)
        
        if name=="Prajwal":
            filename='intruder_{}.jpg'.format(k)
            cv2.imwrite(filename, frame)
            cv2.imshow("Frame", frame)
#             send_email(k)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [7]:
face_recog()

Taking images!!


In [10]:
trig_pin=10
echo_pin=11

In [11]:
board=pymata4.Pymata4()

pymata4:  Version 1.15

Copyright (c) 2020 Alan Yorinks All Rights Reserved.

Opening all potential serial ports...

Waiting 4 seconds(arduino_wait) for Arduino devices to reset...

Searching for an Arduino configured with an arduino_instance = 1


RuntimeError: arduino_instance_id does not match a value on the boards.

In [ ]:
def the_callback(data):
    dist=data[2]
    print("Distance in cm = ",dist)
    if(dist<10):
        take_image()

In [ ]:
board.set_pin_mode_sonar(trig_pin,echo_pin,the_callback)

while True:
    try:
        time.sleep(1)
        board.sonar_read(trig_pin)
    except Exception:
        board.shutdown()

In [ ]:
video_capture.release()
cv2.destroyAllWindows()